<a href="https://colab.research.google.com/github/karthigadDeepsphere/QA_LangChain/blob/main/QA_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "Replace it with OpenAI API Key"
os.getenv('OPENAI_API_KEY')

In [ ]:
!pip install langchain

In [4]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 8.2 MB/s eta 0:00:00


In [ ]:
!pip install openai

In [ ]:
!pip install --upgrade openai

In [6]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [ ]:
!pip install chromadb

In [11]:
#importing Libraries
from langchain.document_loaders import PyPDFLoader
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain

In [12]:
#load document
loaders = ["/content/calPERS-change-retirement-systems.pdf", "/content/Frequently Asked Questions - CalPERS.pdf"]
documents=[]
loader_pdf_list=[]
for loader in loaders:
  loader_pdf = PyPDFLoader(loader)
  loader_pdf_list.append(loader_pdf)
  documents.extend(loader_pdf.load())


In [13]:
def load_aq_chain(querys):
  #Method- 1: Load_qa_chain

  chain = load_qa_chain(llm=OpenAI(), chain_type = "map_reduce")
  query = querys
  # query = "Can I add beneficiary to my retirement plan?"
  return chain.run(input_documents=documents, question=query)


In [14]:
def retrieval_qa(queries):
  #Method- 2: Retrieval QA

  # split the documents into chunks
  text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
  texts = text_splitter.split_documents(documents)
  # select which embeddings we want to use
  embeddings = OpenAIEmbeddings()
  # create the vectorestore to use as the index
  db = Chroma.from_documents(texts, embeddings)
  # expose this index in a retriever interface
  retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})
  # create a chain to answer questions
  qa = RetrievalQA.from_chain_type(
      llm=OpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=True)
  # query = "Can I add beneficiary to my retirement plan?"
  query = queries
  result = qa({"query": query})
  return result


In [15]:
def vectorstore_index_creator(queries):
  #Method- 3: VectorstoreIndexCreator
  index = VectorstoreIndexCreator().from_loaders(loader_pdf_list)
  # query = "Can I add beneficiary to my retirement plan?"
  query = queries
  return index.query(llm=OpenAI(), question=query, chain_type="stuff")

In [33]:
def vectorstore_index_creator_wparams(queries):
  index = VectorstoreIndexCreator(
      text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0),
      embedding=OpenAIEmbeddings(),
      vectorstore_cls=Chroma
  ).from_loaders(loader_pdf_list)
  query = queries
  # query = "Can I add beneficiary to my retirement plan?"
  return index.query(llm=OpenAI(), question=query, chain_type="stuff")


In [17]:
#Method- 4: ConversationalRetrievalChain
def conversational_retrieval_chain():

  # loader = PyPDFLoader("/content/calPERS-change-retirement-systems.pdf")
  # documents=loader.load()
  text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
  texts=text_splitter.split_documents(documents)
  embeddings=OpenAIEmbeddings()
  db=Chroma.from_documents(texts, embeddings)
  retriever=db.as_retriever(search_type="similarity", search_kwargs={"k":2})
  qa = ConversationalRetrievalChain.from_llm(OpenAI(), retriever)
  return qa

In [18]:
def conversational_retrieval_chain_result(queries, follow_question):
  qa = conversational_retrieval_chain()
  chat_history=[]
  # query="Can I add beneficiary to my retirement plan?"
  query = queries
  result_queries=qa({"question":query, "chat_history":chat_history})
  result_queries["answer"]
  chat_history=[(query, result_queries["answer"])]
  # query="who are all can be the beneficiary?"
  query = follow_question
  result_follow=qa({"question":query, "chat_history":chat_history})
  result_follow["answer"]
  return result_queries["answer"], result_follow["answer"]


In [20]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [44]:
print('Question: Can I add beneficiary to my retirement plan?')
print('Following is the response from different method')
print('***********************\n')
query_method_1 = load_aq_chain('Can I add beneficiary to my retirement plan?')
print('Load_aq_chain method:', query_method_1) #load aq chain method
print('\n')
print('***********************\n')
query_method_2 = retrieval_qa('Can I add beneficiary to my retirement plan?')
print('Retrieval_qa method:', query_method_2['result']) #retrieval qa method
print('\n')
print('***********************\n')
query_method_3 = vectorstore_index_creator('Can I add beneficiary to my retirement plan?')
print('Vectorstore Index Creator:', query_method_3) #vectorstore index creator
print('\n')
print('***********************\n')
query_method_3_wparams = vectorstore_index_creator_wparams('Can I add beneficiary to my retirement plan?')
print('Vectorstore Index Creator with params:', query_method_3_wparams) #vectorstore index creator with parameters
print('\n')
print('***********************\n')

Question: Can I add beneficiary to my retirement plan?
Following is the response from different method
***********************

Load_aq_chain method:  Yes, you can add a beneficiary to your retirement plan. It is important to regularly review and update your beneficiaries to ensure that your assets are distributed according to your wishes. Contact your plan administrator for more information on how to add a beneficiary.


***********************

Retrieval_qa method:  Yes, you can add a beneficiary to your retirement plan. However, if you are employed with another publically funded retirement system, CalPERS will not be required to pay the Retired Death Benefit. It is advisable to retire from both systems concurrently if you wish to leave a monthly death benefit to your beneficiary.


***********************

Vectorstore Index Creator:  Yes, you can add a beneficiary to your retirement plan. However, if you pass away after retiring, the lump-sum Retired Death Benefit will only be paid 

In [45]:
# method - 4 conversational retrieval chain takes two argument, one is questions and other is the follow-up question to demonstrate
# chat history
query_method_4, follow_answer = conversational_retrieval_chain_result('Can I add beneficiary to my retirement plan?', 'who are all can be the beneficiary?')
print('Conversational Retrieval Chain Result:', query_method_4)
print('Follow-up question:who are all can be the beneficiary?')
print('conversational Retrieval Chain Result Follow-up answer:', follow_answer)

Conversational Retrieval Chain Result:  Yes, you can add a beneficiary to your retirement plan, but if you become ill and wish to leave a monthly death benefit to anyone, you must retire from CalPERS and choose an option that provides a post-retirement monthly death benefit. Additionally, it is advisable to retire from both systems concurrently to ensure the post-retirement death benefits are available.
Follow-up question:who are all can be the beneficiary?
conversational Retrieval Chain Result Follow-up answer:  The beneficiary for your retirement plan can be anyone you choose, such as a spouse, child, or other designated individual.
